In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from osgeo import gdal, osr
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import scipy.io as sio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as matplotlib
import colorcet as cc
from matplotlib.colors import LinearSegmentedColormap

CA_keras_mse = sio.loadmat('CA_keras_adam_mse.mat')  
seasonal_wave = np.squeeze(np.array(CA_keras_mse['seasonal_wave']))
interann_wave = np.squeeze(np.array(CA_keras_mse['interann_wave']))
tide_amp = np.squeeze(np.array(CA_keras_mse['tide_amp']))
residual_std = np.squeeze(np.array(CA_keras_mse['residual_std']))
max_seas_doy = np.squeeze(np.array(CA_keras_mse['max_seas_doy']))
peak_time = np.squeeze(np.array(CA_keras_mse['peak_time']))
lons = np.array(llon_1)[:,0]
lats = np.array(llon_1)[:,1]

CA_keras_slopes_mse = sio.loadmat('CA_keras_slopes_adam_mse_historical_MB.mat')  
llon_1 = np.squeeze(np.array(CA_keras_slopes_mse['llon_1']))
llon_2 = np.squeeze(np.array(CA_keras_slopes_mse['llon_2']))
orient = np.squeeze(np.array(CA_keras_slopes_mse['orient']))
rmse = np.squeeze(np.array(CA_keras_slopes_mse['rmse']))
rmse_weighted = np.squeeze(np.array(CA_keras_slopes_mse['rmse_weighted']))
bias = np.squeeze(np.array(CA_keras_slopes_mse['bias']))

states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='10m',
    facecolor='none')

# Define the locations with their names and coordinates
locations = {
    "Torrey Pines": (32.9369, -117.2613),
    "Newport Beach": (33.606137, -117.917086),
    "Santa Monica": (34.0195, -118.4912),
    "Santa Barbara": (34.4208, -119.6982),
    "Ragged Point": (35.780691, -121.331689),
    "Monterey": (36.6002, -121.8947),
    "Ocean Beach": (37.7597, -122.5101),
    "Bodega Bay": (38.3333, -123.0481),
    "Mendocino": (39.3077, -123.7995),
    "King Range": (40.0214, -124.0734),
    "Eureka": (40.8021, -124.1637),
    "Crescent City": (41.7558, -124.2026)
}
santa_barbara_lat = locations["Ragged Point"][0]

# Create a map
fig, ax = plt.subplots(figsize=(7, 20), subplot_kw={'projection': ccrs.PlateCarree()})

# Set extent for California
ax.set_extent([-124, -114, 31, 43], crs=ccrs.PlateCarree())

# Add features
ax.add_feature(cfeature.COASTLINE,linewidth=0.2)
# ax.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.2)
ax.add_feature(cfeature.LAND, facecolor='white',linewidth=0.2)
# ax.add_feature(cfeature.STATES, linestyle=':',linewidth=0.2)

# Define arrow style
arrow_style = {
    'arrowprops': {
        'arrowstyle': '<-',
        'lw': 1,
        'color': 'black'
    }
}

gdal.UseExceptions()
fname = 'CA_DEM/output_SRTM15Plus.tif'
ds = gdal.Open(fname)
gt = ds.GetGeoTransform()
proj = ds.GetProjection()

def plot_ca_beach(ax,lons,lats,pdata,orient,ds,gt,states_provinces,colorbar=True,cbar_label='DNN Beach Slope',clim=None,cmap_='viridis',txt=None, y_min=32.3, y_max=42.5, x_min=-125.5, x_max=-116.5):
    data = ds.ReadAsArray()
    extent = (gt[0], gt[0] + ds.RasterXSize * gt[1],gt[3] + ds.RasterYSize * gt[5], gt[3])
    img = ax.imshow(data, extent=extent, origin='upper',cmap='gray',vmin=-20000,vmax=1500)
    ax.coastlines(resolution='10m',linewidth=0.5)
    # ax.add_feature(states_provinces, edgecolor='gray')
    # ax.add_feature(cfeature.BORDERS)
    ax.set_ylim([y_min,y_max])
    ax.set_xlim([x_min,x_max])
    idx = np.isnan(pdata)
    color = pdata[~idx]
    length = np.ones_like(lats)*0.05 # length of vectors
    if clim==None:
        clim=(np.nanmin(pdata),np.nanmax(pdata))
    cx0 = ax.quiver(lons[~idx],lats[~idx],length[~idx],length[~idx],color,angles=orient[~idx],pivot='middle',scale=1,
               headlength=0,headwidth=0,headaxislength=0,cmap=cmap_,clim=clim)
    if colorbar:
        axins1 = inset_axes(
            ax,
            width="3%",  # width: 50% of parent_bbox width
            height="30%",  # height: 5%
            loc="center left",
            bbox_to_anchor=(0.0,-0.24,1,1), bbox_transform=ax.transAxes
        )
        axins1.xaxis.set_ticks_position("bottom")
        cbar = fig.colorbar(cx0, cax=axins1, orientation="vertical")
        cbar.set_label(cbar_label, fontsize=12)
        cbar.ax.tick_params(labelsize=12)
        if txt:
            ax.text(-125.4,32.6,txt,fontsize=12)
    return 


from scipy.ndimage import gaussian_filter1d
# Define the slightly lighter dark gray color
lighter_dark_gray = [0.36, 0.36, 0.36, 1]  # RGBA for lighter dark gray
# Get the original 'coolwarm' colormap
original_coolwarm_colors = plt.cm.coolwarm(np.linspace(0, 1, 512))
# Create the transitions to and from the 'coolwarm' colormap edges
transition_to_cool_lighter = np.linspace(lighter_dark_gray, original_coolwarm_colors[0], 128)
transition_from_warm_lighter = np.linspace(original_coolwarm_colors[-1], lighter_dark_gray, 128)
# Concatenate the transitions with the original 'coolwarm' colormap
new_colors_lighter_dark_gray = np.vstack((transition_to_cool_lighter, original_coolwarm_colors, transition_from_warm_lighter))
# first_half, second_half = np.split(new_colors_lighter_dark_gray, 2, axis=0)
# # Concatenate the second half followed by the first half
# new_colors_lighter_dark_gray = np.concatenate((second_half, first_half), axis=0)
# Apply Gaussian smoothing with a specified sigma value
sigma = 30  # The sigma value for smoothing
smoothed_colors_lighter_dark_gray = gaussian_filter1d(new_colors_lighter_dark_gray, sigma=sigma, axis=0, mode='nearest')
# Create a new LinearSegmentedColormap from the smoothed colors
smoothed_lighter_dark_gray_transition_cmap = LinearSegmentedColormap.from_list(
    'SmoothedLighterDarkGrayTransitionCoolwarm', smoothed_colors_lighter_dark_gray)

def convert_day_of_year(day_of_year_array):
    OCT_1_DAY_OF_YEAR = 274
    return np.where(day_of_year_array < OCT_1_DAY_OF_YEAR, 
                    day_of_year_array + (365 - OCT_1_DAY_OF_YEAR), 
                    day_of_year_array - OCT_1_DAY_OF_YEAR)

pdata = convert_day_of_year(np.array(max_seas_doy))
plot_ca_beach(ax,lons,lats,pdata,orient,ds,gt,states_provinces,colorbar=True,clim=(0,365),cmap_=smoothed_lighter_dark_gray_transition_cmap,txt='Time of Minimum\nBeach Width',cbar_label='Day of Water Year\n(Day 0 = October 1)')

for name, (lat, lon) in locations.items():
    # Different style if the location is south of Santa Barbara
    ax.scatter(lon,lat,marker='.',color='k',s=5)
ax.text(-125.4,42.2,'(a)',fontsize=12)  
plt.savefig('figures/map_width_min.pdf',bbox_inches='tight')

plt.show()

In [ ]:
CA_keras_along_dist = sio.loadmat('CA_keras_along_dist.mat')
along_dist_transect = np.squeeze(CA_keras_along_dist['along_dist_transect'])
closest_distance = np.squeeze(CA_keras_along_dist['closest_distance'])
along_dist_names = np.squeeze(CA_keras_along_dist['along_dist_names'])
along_dist_names_socal = np.squeeze(CA_keras_along_dist['along_dist_names_socal'])

In [ ]:
def smooth_data(x, y, smoothing_distance, cutoff=100):
    # Initialize the smoothed data array
    smoothed_y = np.empty_like(y)
    # Calculate the distances once since the x array is static
    dist_matrix = np.abs(x[:, np.newaxis] - x)
    # Apply the cutoff to the distances
    dist_matrix[dist_matrix > cutoff] = np.inf
    # Calculate the weights using a Gaussian function, vectorized for all distances
    weights = np.exp(-0.5 * (dist_matrix / smoothing_distance) ** 2)
    # Ensure weights are zero where distances exceed the cutoff
    weights[dist_matrix > cutoff] = 0
    # Compute the weighted average, also vectorized
    smoothed_y = np.nansum(weights * y, axis=1) / np.nansum(weights, axis=1)
    return smoothed_y

window_size = 100

def plot_along_ca_beach(ax, along_dist_transect, closest_distance, pdata, along_dist_names, txt=None, ylabel=None, ylim=[-25, 5],cbar_label='Beach\nSlope',yticks=False,window_size = 3,nosmooth=False,legend=False):
    pdata[closest_distance > 0.8] = np.nan
    pdata[np.gradient(along_dist_transect) > 10] = np.nan
    smoothed_pdata = smooth_data(along_dist_transect, pdata, 0.25)
    smoothed_pdata[closest_distance > 0.8] = np.nan
    smoothed_pdata[np.gradient(along_dist_transect) > 10] = np.nan
    for x_value in along_dist_names:
        ax.axhline(y=x_value, color='k', linestyle='-',linewidth=0.5)
    
    if nosmooth:
        smoothed_pdata = pdata

    ax.plot(smoothed_pdata, along_dist_transect, color=[0.5, 0.5, 0.5], alpha=0.1)
    ax.fill_betweenx(along_dist_transect, 0, smoothed_pdata, where=(smoothed_pdata >= 0), color=sns.color_palette("Paired")[4],label=cbar_label)
    ax.fill_betweenx(along_dist_transect, smoothed_pdata, 0, where=(smoothed_pdata < 0), color=sns.color_palette("Paired")[0])
    ax.set_xlim(ylim)
    ax.set_ylim([0, 1675])
    ax.grid(axis='x')
    # ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')
    if not yticks:
        ax.set_yticks([])
        ax.set_yticklabels([])
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.set_xlabel(ylabel, fontsize=12)

    if txt:
        ax.text(31.9, -125.4, txt, fontsize=12, rotation=90)
    return 

# Create a figure with 6 rows and 1 column of subplots
fig, (ax0,ax1,ax2,ax3,ax4) = plt.subplots(nrows=1,ncols=5,figsize=(10,8))
plt.subplots_adjust(wspace=0.125)
plt.subplots_adjust(hspace=0.0)

pdata = shore_winterspring - shore_summerfall
plot_along_ca_beach(ax0,along_dist_transect,closest_distance,pdata,along_dist_names,ylabel='Shoreline\nChange\nBetween\nDec-May\nand\nJun-Nov (m)',ylim=[-50,25])
ax0.text(-48,1635,'(b)',fontsize=12)  

pdata = shore_95_winterspring - shore_95_summerfall
plot_along_ca_beach(ax1,along_dist_transect,closest_distance,pdata,along_dist_names,ylabel='Change in Shoreline\nStandard Deviation\nBetween\nDec-May\nand\nJun-Nov (m)',ylim=[-30,30])
ax1.text(-29,1635,'(c)',fontsize=12)  

pdata = np.array(seasonal_wave)
plot_along_ca_beach(ax2,along_dist_transect,closest_distance,pdata,along_dist_names,ylabel='Seasonal\nShoreline\nChange\n(m)',ylim=[0,85])
ax2.text(66,1635,'(d)',fontsize=12)  

pdata = np.array(residual_std) + np.array(interann_wave)
plot_along_ca_beach(ax3,along_dist_transect,closest_distance,pdata,along_dist_names,ylabel='Interannual and\nSub-seasonal\nShoreline Change\n(m)',ylim=[0,85])
ax3.text(66,1635,'(e)',fontsize=12)  

CA_keras_slopes_mse = sio.loadmat('CA_keras_seas_slopes.mat')  
slope = np.squeeze(np.array(CA_keras_slopes_mse['slope_CA_MSL_MHHW']))
slope[np.isnan(tide_amp)] = np.NaN

pdata = np.array(slope)
plot_along_ca_beach(ax4,along_dist_transect,closest_distance,pdata,along_dist_names,ylabel='Beach Slope',ylim=[0,0.15],yticks=False)
ax4.text(0.122,1635,'(f)',fontsize=12)  

ax4.set_ylabel('Along-Coast Distance (km)',fontsize=12)
ax4.yaxis.tick_right()
ax4.yaxis.set_label_position('right')

plt.savefig('figures/along_beach_transects.pdf',bbox_inches='tight')